# Phase 6: LSTM Model Inference - Smart Parking IoT System

## Overview
This notebook loads the best LSTM model and performs inference on the entire smart parking dataset.

### Objectives
- Load the best trained LSTM model
- Perform inference on the complete dataset
- Add predicted occupancy rate column
- Identify train/test splits for each row
- Analyze prediction quality and coverage

### Model Information
- Best LSTM Model: Loaded from Phase 5
- Target: Occupancy Rate (0-1)
- Input: 24-hour sequence of features
- Output: 1-hour ahead prediction

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
import pickle
from datetime import datetime, timedelta

# Deep learning libraries
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import joblib

# Set display options
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Libraries imported successfully!")
print(f"TensorFlow version: {tf.__version__}")

In [ ]:
# Load the complete dataset
print("🔄 Loading complete smart parking dataset...")

# Define data path
data_path = Path(r"C:\Users\vedp3\OneDrive\Desktop\AAI_530_Final_Project\AAI530-Group10-smart-parking-iot-forecasting\data\raw\smart_parking_full.csv")

# Load the dataset
df = pd.read_csv(data_path, sep=';')
print("✅ Complete dataset loaded successfully!")
print(f"📊 Dataset Shape: {df.shape}")
print(f"   - Rows: {df.shape[0]:,}")
print(f"   - Columns: {df.shape[1]}")

In [ ]:
# Load the best LSTM model and components
print("🔄 Loading best LSTM model and components...")

# Define models directory
models_dir = Path(r"C:\Users\vedp3\OneDrive\Desktop\AAI_530_Final_Project\AAI530-Group10-smart-parking-iot-forecasting\models")

# Load model metadata
with open(models_dir / "best_lstm_metadata.pkl", 'rb') as f:
    lstm_metadata = pickle.load(f)

print(f"📋 Model Metadata:")
print(f"   Model Name: {lstm_metadata['model_name']}")
print(f"   Model Type: {lstm_metadata['model_type']}")
print(f"   Sequence Length: {lstm_metadata['sequence_length']}")
print(f"   Forecast Horizon: {lstm_metadata['forecast_horizon']}")
print(f"   Target Segment: {lstm_metadata['target_segment']}")
print(f"   Performance R²: {lstm_metadata['performance_metrics']['r2']:.4f}")

# Load the best LSTM model
model_path = models_dir / f"best_{lstm_metadata['model_name']}_model.h5"
best_model = load_model(model_path)
print(f"✅ Best LSTM model loaded: {model_path.name}")

# Load scalers
scaler_features = joblib.load(models_dir / "lstm_feature_scaler.pkl")
scaler_target = joblib.load(models_dir / "lstm_target_scaler.pkl")
print("✅ Feature and target scalers loaded")

# Extract parameters
SEQUENCE_LENGTH = lstm_metadata['sequence_length']
FORECAST_HORIZON = lstm_metadata['forecast_horizon']
features = lstm_metadata['features']
target_segment = lstm_metadata['target_segment']

In [ ]:
# Preprocess the complete dataset
print("🔧 Preprocessing complete dataset for inference...")

# Convert timestamp and sort
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values('timestamp')

# Calculate occupancy rate
df['occupancy_rate'] = df['occupied'] / df['capacity']
df['occupancy_rate'] = df['occupancy_rate'].clip(0, 1)

# Create time-based features
df['hour'] = df['timestamp'].dt.hour
df['day_of_week'] = df['timestamp'].dt.dayofweek
df['month'] = df['timestamp'].dt.month
df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
df['is_rush_hour'] = ((df['hour'].between(7, 9)) | (df['hour'].between(16, 18))).astype(int)

# Create cyclical features
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
df['day_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
df['day_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)

# Aggregate data by hour for consistency with training
print("📊 Aggregating data by hour...")
df_hourly = df.groupby([pd.Grouper(key='timestamp', freq='H'), 'segmentid']).agg({
    'occupancy_rate': 'mean',
    'capacity': 'mean',
    'hour': 'first',
    'day_of_week': 'first',
    'month': 'first',
    'is_weekend': 'first',
    'is_rush_hour': 'first',
    'hour_sin': 'first',
    'hour_cos': 'first',
    'day_sin': 'first',
    'day_cos': 'first'
}).reset_index()

# Filter for target segment
df_target = df_hourly[df_hourly['segmentid'] == target_segment].copy()
df_target = df_target.sort_values('timestamp')

# Prepare features
df_inference = df_target[features].copy()
df_inference = df_inference.fillna(df_inference.median())

print(f"✅ Dataset preprocessed for inference:")
print(f"   Shape: {df_inference.shape}")
print(f"   Time range: {df_target['timestamp'].min()} to {df_target['timestamp'].max()}")
print(f"   Features: {features}")

In [ ]:
# Create sequences for inference
def create_inference_sequences(data, sequence_length):
    """
    Create sequences for LSTM inference
    """
    sequences = []
    
    for i in range(len(data) - sequence_length + 1):
        sequences.append(data[i:(i + sequence_length)])
    
    return np.array(sequences)

print("🔧 Creating sequences for inference...")

# Scale features
features_scaled = scaler_features.transform(df_inference[features])

# Create sequences
X_inference = create_inference_sequences(features_scaled, SEQUENCE_LENGTH)

print(f"✅ Inference sequences created:")
print(f"   Input sequences shape: {X_inference.shape}")
print(f"   Total predictions possible: {len(X_inference)}")

In [ ]:
# Perform inference on all sequences
print("🚀 Performing LSTM inference on all sequences...")

# Make predictions
predictions_scaled = best_model.predict(X_inference, verbose=0)

# Inverse transform predictions
predictions = scaler_target.inverse_transform(predictions_scaled)

print(f"✅ Inference completed:")
print(f"   Predictions shape: {predictions.shape}")
print(f"   Prediction range: {predictions.min():.4f} to {predictions.max():.4f}")
print(f"   Mean prediction: {predictions.mean():.4f}")

In [ ]:
# Create final dataset with predictions and train/test labels
print("📊 Creating final dataset with predictions...")

# Create results dataframe
results_df = df_target.copy()

# Add predictions (align with original data)
# Predictions start from sequence_length index
predictions_flat = predictions.flatten()
prediction_indices = range(SEQUENCE_LENGTH, SEQUENCE_LENGTH + len(predictions_flat))

# Initialize prediction column with NaN
results_df['predicted_occupancy_rate'] = np.nan

# Assign predictions to correct positions
for i, idx in enumerate(prediction_indices):
    if idx < len(results_df):
        results_df.iloc[idx, results_df.columns.get_loc('predicted_occupancy_rate')] = predictions_flat[i]

# Define train/test split (80/20 split like in training)
split_point = int(len(results_df) * 0.8)

# Add train/test labels
results_df['data_split'] = 'train'
results_df.iloc[split_point:, results_df.columns.get_loc('data_split')] = 'test'

# Calculate prediction metrics where available
valid_predictions = results_df.dropna(subset=['predicted_occupancy_rate'])

if len(valid_predictions) > 0:
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    
    mse = mean_squared_error(valid_predictions['occupancy_rate'], valid_predictions['predicted_occupancy_rate'])
    mae = mean_absolute_error(valid_predictions['occupancy_rate'], valid_predictions['predicted_occupancy_rate'])
    r2 = r2_score(valid_predictions['occupancy_rate'], valid_predictions['predicted_occupancy_rate'])
    
    print(f"📈 Prediction Quality Metrics:")
    print(f"   MSE: {mse:.6f}")
    print(f"   MAE: {mae:.6f}")
    print(f"   R²: {r2:.4f}")
    print(f"   Valid predictions: {len(valid_predictions):,}")

print(f"\n📊 Final Dataset Summary:")
print(f"   Total rows: {len(results_df):,}")
print(f"   Training rows: {len(results_df[results_df['data_split'] == 'train']):,}")
print(f"   Testing rows: {len(results_df[results_df['data_split'] == 'test']):,}")
print(f"   Rows with predictions: {len(valid_predictions):,}")
print(f"   Rows without predictions: {len(results_df) - len(valid_predictions):,}")

# Display sample of results
print(f"\n📋 Sample Results:")
display_cols = ['timestamp', 'segmentid', 'occupancy_rate', 'predicted_occupancy_rate', 'data_split']
display(results_df[display_cols].head(10))

In [ ]:
# Visualize predictions vs actual values
print("📈 Visualizing predictions...")

# Create visualization data
viz_data = results_df.dropna(subset=['predicted_occupancy_rate']).copy()

if len(viz_data) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(18, 12))
    fig.suptitle('LSTM Model Predictions - Complete Dataset Analysis', fontsize=16, fontweight='bold')
    
    # Plot 1: Time series comparison (sample)
    sample_size = min(500, len(viz_data))
    sample_data = viz_data.head(sample_size)
    
    axes[0,0].plot(sample_data['timestamp'], sample_data['occupancy_rate'], 
                    label='Actual', color='blue', linewidth=2, alpha=0.8)
    axes[0,0].plot(sample_data['timestamp'], sample_data['predicted_occupancy_rate'], 
                    label='Predicted', color='red', linewidth=1.5, alpha=0.8)
    axes[0,0].set_title('Predictions vs Actual (First 500 points)')
    axes[0,0].set_xlabel('Time')
    axes[0,0].set_ylabel('Occupancy Rate')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)
    
    # Plot 2: Scatter plot
    axes[0,1].scatter(viz_data['occupancy_rate'], viz_data['predicted_occupancy_rate'], 
                      alpha=0.5, s=10)
    min_val = min(viz_data['occupancy_rate'].min(), viz_data['predicted_occupancy_rate'].min())
    max_val = max(viz_data['occupancy_rate'].max(), viz_data['predicted_occupancy_rate'].max())
    axes[0,1].plot([min_val, max_val], [min_val, max_val], 'r--', alpha=0.8)
    axes[0,1].set_title('Predictions vs Actual (Scatter)')
    axes[0,1].set_xlabel('Actual Occupancy Rate')
    axes[0,1].set_ylabel('Predicted Occupancy Rate')
    axes[0,1].grid(True, alpha=0.3)
    
    # Plot 3: Residuals
    residuals = viz_data['occupancy_rate'] - viz_data['predicted_occupancy_rate']
    axes[1,0].hist(residuals, bins=30, alpha=0.7, edgecolor='black')
    axes[1,0].axvline(x=0, color='red', linestyle='--', alpha=0.8)
    axes[1,0].set_title('Prediction Residuals Distribution')
    axes[1,0].set_xlabel('Residuals')
    axes[1,0].set_ylabel('Frequency')
    axes[1,0].grid(True, alpha=0.3)
    
    # Plot 4: Train/Test split visualization
    train_data = results_df[results_df['data_split'] == 'train'].dropna(subset=['predicted_occupancy_rate'])
    test_data = results_df[results_df['data_split'] == 'test'].dropna(subset=['predicted_occupancy_rate'])
    
    axes[1,1].scatter(train_data['occupancy_rate'], train_data['predicted_occupancy_rate'], 
                      alpha=0.5, s=8, color='blue', label='Train')
    axes[1,1].scatter(test_data['occupancy_rate'], test_data['predicted_occupancy_rate'], 
                      alpha=0.5, s=8, color='red', label='Test')
    
    axes[1,1].plot([min_val, max_val], [min_val, max_val], 'k--', alpha=0.8)
    axes[1,1].set_title('Train vs Test Performance')
    axes[1,1].set_xlabel('Actual Occupancy Rate')
    axes[1,1].set_ylabel('Predicted Occupancy Rate')
    axes[1,1].legend()
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ No valid predictions to visualize")

In [ ]:
# Save the complete inference results
print("💾 Saving inference results...")

# Create output directory
output_dir = Path(r"C:\Users\vedp3\OneDrive\Desktop\AAI_530_Final_Project\AAI530-Group10-smart-parking-iot-forecasting\data\inference")
output_dir.mkdir(exist_ok=True)

# Save complete results
output_path = output_dir / "lstm_inference_complete_results.csv"
results_df.to_csv(output_path, index=False)

# Save only rows with predictions
predictions_only_path = output_dir / "lstm_predictions_only.csv"
valid_predictions.to_csv(predictions_only_path, index=False)

# Save inference metadata
inference_metadata = {
    'model_name': lstm_metadata['model_name'],
    'model_type': 'LSTM',
    'inference_timestamp': datetime.now().isoformat(),
    'total_rows': len(results_df),
    'train_rows': len(results_df[results_df['data_split'] == 'train']),
    'test_rows': len(results_df[results_df['data_split'] == 'test']),
    'prediction_rows': len(valid_predictions),
    'sequence_length': SEQUENCE_LENGTH,
    'target_segment': target_segment,
    'performance_metrics': {
        'mse': float(mse) if len(valid_predictions) > 0 else None,
        'mae': float(mae) if len(valid_predictions) > 0 else None,
        'r2': float(r2) if len(valid_predictions) > 0 else None
    }
}

with open(output_dir / "inference_metadata.pkl", 'wb') as f:
    pickle.dump(inference_metadata, f)

print(f"✅ Results saved successfully:")
print(f"   Complete results: {output_path}")
print(f"   Predictions only: {predictions_only_path}")
print(f"   Metadata: {output_dir / 'inference_metadata.pkl'}")

print(f"\n📊 Final Summary:")
print(f"   🎯 Model: {lstm_metadata['model_name']}")
print(f"   📈 Total predictions: {len(valid_predictions):,}")
print(f"   🏷️ Dataset split: {len(results_df[results_df['data_split'] == 'train']):,} train / {len(results_df[results_df['data_split'] == 'test']):,} test")
print(f"   📊 Prediction quality: R² = {r2:.4f}")
print(f"   💾 Output saved to: {output_dir}")

print(f"\n✅ LSTM Inference Complete!")
print(f"🚀 Ready for production deployment!")